In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# 1 A Notch Digital Filter

A notch filter is a filter that passes almost all frequencies with unit amplitude, except for a narrow
range of frequencies centered on the rejection frequency $f_0$ (at which frequency nothing is passed).

A simple notch digital filter has a z-transform

$$ W(z) = MF(z) F(z^*)^* = M \frac{z - q}{z - p} \frac{z - q^*}{z - p^*} $$

where $M$ is a normalization factor, $q = e^{-i 2 \pi f_0 / f_s}$, $p = (1 + \epsilon) q$, $f_s$ 
is the sampling rate ($f_s = 1 / \Delta$) and $\epsilon$ is a small positive number ($0 < \epsilon \ll 1$).

 1. What are the poles and zeros of this system? Where are they located with respect to the unit
    circle? Is this a stable system? Is this filter real (i.e., given real input, does it produce real
    output)?
 2. Given $f_s = 12 \text{ cycles/year}$, $f_0 = 1 \text{ cycle/year}$, $M = 1.0546$ and $\epsilon = 0.053783$, plot the
    power spectrum $|W(f)|^2$ (i.e., square of amplitude spectrum).
 3. What is the full-width-half-max (FWHM) value $f_\text{fwhm}$ of this notch filter? Which parameter(s)
    should you change to make the notches sharper, i.e., $f_\text{fwhm}$ smaller?

_Hint_: For question 2, $W(f)$ is obtained by $W(z = e^{-i \omega \Delta}$). For question 3, you don't have to
compute the FWHM value analytically (although it can be done); an inspection of the discrete values of $|W(f)|^2$
vector is sufficient. Note here $f_\text{fwhm}$ is in terms of frequency (1/year), not angular frequency.

# 2 Implementing the Notch Filter

Notch filter introduced in section 1 can be written out fully as

$$ W(z) = \frac{N(z)}{D(z)} = \frac{a + bz + cz^2}{1 + Bz + Cz^2} $$

A filter of the form above can be implemented as two filters in succession: first the 'numerator filter'
`[a b c]` as a 3-term direct convolution, then the 'denominator filter' as the inverse filter of the 3-term
filter `[1 B C]` by feedback filtering.

 1. What are the values of a; b; c; B; C for the notch filter defined in question 1.2?
 2. Write a function for a general rational digital filter with numerator and denominator
    coefficients N and D which produces the filtered time series $y$ for a given input $x$,
    `y = ratFilter(N,D,x)`.
 3. Use `ratFilter` function to determine the impulse response of this notch filter (i.e., the output
    of this filter when the input is a discrete delta function). Speculate on how the impulse
    response would change if we half the $f_\text{fwmh}$ value.
 4. Fourier transform the impulse response to obtain the frequency response $|W(f)|$
    of this notch filter. Plot it on top of the theoretical spectrum calculated based on the z-transform.

# 3 The Mauna Loa $CO_2$ Data

The file `co2data.py` (on the web site) contains carbon dioxide values (in parts per million) measured
every month at the Mauna Loa Observatory in Hawaii, from January 1965 to December 2003.
They show clearly a rising trend in atmospheric $CO_2$. The trend is overlaid with a strong annual
oscillation. Your job is to remove the annual oscillation and display the trend more clearly. There
are two possible approaches: (a) you could apply your notch filter to the series to remove this
annual variation, or (b) you could Fourier transform it with `fft`, remove the annual variation by
setting the spectrum at appropriate frequencies to zero, and transform back to the time domain with
`ifft`.

Write code to accomplish the following:

 1. Before applying the filters, it is helpful to remove the trend of the signal using numpy
    function `polyfit`. Fit a straight line to your data and then detrend your data by removing the
    straight line. Plot both the original data and the detrended data.
 2. Apply your notch filter to the detrended data and add back the trend.
 3. FT the detrended data into the frequency domain, and plot both its amplitude and phase
    spectrum. Make another plot that zooms in at $f = [0, 2.5] \text{ cycles per year}$. Now set the
    Fourier spectrum corresponding to frequencies beyond $0.9 \text{ cycles per year}$ to zero, which
    effectively removes the annual oscillation. Transform the spectrum back to time domain and
    add back the trend.
 4. Now plot the original data, the notch-filtered data from part 2 and f-domain filtered data
    from part 3 on top of each other with different colors. Which method gives more satisfactory
    result? But can you think of any advantages/disadvantages in using either method?
 5. Now try redo part 2, 3, and 4 with the original data, not the detrended data. Of course you
    don't need to add back the trend after filtering any more. Display your results and comment
    on the importance of detrending before applying the filters.

# 4 Bonus question: Butterworth filter (2 marks)

**Note: Please only do this part if you have extra time.**

Low-pass Butterworth filter is very commonly used to filter out high-frequency noise in digital
data. It has a power spectrum in the analytical form of

$$ |H(\omega)|^2 = \frac{1}{1 + \left( \frac{\omega}{\omega_0} \right)^{2N}} $$

where $\omega_0$ is the reference angular frequency below which input signal will be low-pass filtered
through.

A common practice of converting the power spectrum response (in $\omega$ domain) into a rational
filter (in $z$ domain) is to use the bilinear transformation:

$$ j \omega t = 2 \frac{1 - z}{1 + z} $$

For simplicity, let's consider a second-order Butterworth low-pass filter, i.e. $N = 2$.

 1. If we let $S(z) = H(z) H^*(1/z^*)$, prove that $|H(\omega)|^2 = S(z) \Big|_{z=e^{-i \omega \Delta t}}$.

    Hint: Note here $H^*(1-z^*)$ is just an abstract way of saying its poles and zeros are conjugate
    reciprocal pairs of those of $H(z)$. Here we use $H(\omega)$ to denote the amplitude spectrum, i.e.
    $H(\omega) = H(z) \Big|_{e^{i \omega \Delta}}$.

 2. Derive the expression for $S(z)$ through bilinear transformation of (3). Write it in terms of
    division of a numerator polynomial of $z$ over a denominator polynomial of $z$.
 3. Factor $H(z)$ out from $S(z)$ by picking the right poles and zeros. Write $H(z)$ in the form of
    $$ H(z) = \frac{a_0 + a_1 z + a_2 z^2}{1 + b_1 z + b_2 z^2} $$
    and give the expressions for $a_0$, $a_1$, $a_2$, $b_1$, $b_2$ in terms of $\omega_0$ and
    $\Delta t$. Bear in mind that for $H(z)$ to be a stable filter, all of its poles should be outside the unit circle.

    _Hint_: To make your derivation easier, you may want to define some auxiliary variables along the way:
    $d = \frac{\omega_0 \Delta t}{2}$, $e_{\pm} = \frac{1 \pm i}{\sqrt{2}}$, and use the fact

    $$ A^4 + B^4 = (A + e_{-} B)(A - e_{-} B)(A + e_{+} B)(A - e_{+} B) $$

 4. Set $f_0 = 1 / 20 \text{ Hz}$ ($\omega_0 = 2 \pi f_0$), using `ratFilter()`, apply this low-pass Butterworth filter
    to the data $x(t)$ from Lab 1 (`RAYN.II.LHZ.sem`) and plot the filtered time series $y(t)$ on
    top of the original time series $x(t)$. Does this filter preserve the exact phase of the original
    time series?
 5. Now apply the same filter to the _time-reversed_ $y(t)$ and then _time-reverse_ the filtered result
    (i.e. applying the Butterworth filter twice to $x(t)$, forward and then backward). Plot this
    result against the original time series $x(t)$. Is the phase preserved in this case? Why?